# Notebook para aprendizado de mongo

Esse notebook serve unicamente para eu aprender as funções básicas do mongo antes de colocá-lo em um projeto, para isso farei um CRUD em uma collection, além de outras manipulações básicas.

### Instalações e imports necessários

In [60]:
! pip install pymongo
from pymongo import MongoClient
import csv
from pprint import pprint

### Conexão

A url é na verdade a connection string provida pelo MongoDBCompass, ainda devo ter o trabalho de criar um cluster na nuvem para trabalhar com isso de um jeito mais correto, mas por enquanto escolho deixar isso em aberto e focar no CRUD em sí.

Já a célula seguinte ocorre o passo a passo da conexção com uma collection. Primeiro entra com o cliente, depois em um banco de dados dentro desse cliente e por fim no banco de dados é possível acessar a collection.

Salvei os dados que eu quero fazer upload em um csv. Nesse exato momento é 00h18 e eu estou sem criatividade, então essa coleção teste será sobre algo um tanto estranho: as letras do teclado do meu notebook. 

In [13]:
url = 'mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass%20Community&ssl=false'

In [106]:
client = MongoClient(url)
db = client.get_database('computador')
col = db.get_collection('teclado')

In [130]:
teclado = []
tecla = {}

with open('board_data.csv', 'r', encoding='utf-8-sig') as data:
    reader = csv.reader(data)
    keys = next(reader)
    for row in reader:
        tecla = {}
        for tup in zip(keys, row):
            if tup[1] != '':
                try:
                    tecla.update({tup[0]:float(tup[1])})
                except ValueError:
                    tecla.update({tup[0]:tup[1]})
        teclado.append(tecla)    

### Create
Primeiro me foi um tanto estranho ter duas funções diferentes para isso, mas até que faz sentido pra ser mais flexível dependendo do caso.

A primeira insere apenas um, enquanto que a segunda está inserindo todos os outros.

In [131]:
col.insert_one(
    teclado[0]
)

col.insert_many(
    teclado[1:len(teclado)]
)

### Read

In [109]:
# pega todos os documentos dessa coleção
cur = col.find({})
for doc in cur:
    print(doc)

{'_id': ObjectId('5f090ccb7fdd86351b5840a6'), 'name': 'ze', 'type': 'letter', 'character': 'z', 'shift': 'Z', 'common_action': 'writes letter z', 'size_x': '1', 'size_y': '1'}
{'_id': ObjectId('5f090ccb7fdd86351b5840a7'), 'name': 'xis', 'type': 'letter', 'character': 'x', 'shift': 'X', 'common_action': 'writes letter x', 'size_x': '1', 'size_y': '1'}
{'_id': ObjectId('5f090ccb7fdd86351b5840a8'), 'name': 'ce', 'type': 'letter', 'character': 'c', 'shift': 'C', 'common_action': 'writes letter c', 'size_x': '1', 'size_y': '1'}
{'_id': ObjectId('5f090ccb7fdd86351b5840a9'), 'name': 'comma', 'type': 'punctuation mark', 'character': ',', 'shift': '<', 'common_action': 'writes a comma', 'description': 'both characters on the left, comma lower than symbol of less than ', 'size_x': '1', 'size_y': '1'}
{'_id': ObjectId('5f090ccb7fdd86351b5840aa'), 'name': 'shift', 'type': 'command', 'common_action': 'Change the size of the letters or use the top character', 'size_x': '3', 'size_y': '1'}
{'_id': Ob

In [83]:
# só traz o primeiro, caso tenham mais com esse nome poderia ser um problema
cur = col.find({'name':'comma'})
pprint(cur.next())

{'_id': ObjectId('5f090ccb7fdd86351b5840a9'),
 'character': ',',
 'common_action': 'writes a comma',
 'description': 'both characters on the left, comma lower than symbol of less '
                'than ',
 'name': 'comma',
 'shift': '<',
 'size_x': '1',
 'size_y': '1',
 'type': 'punctuation mark'}


In [114]:
# $ne é um operador para 'not equal' existem outros que eu vou explorar depois já que parece algo bem básico e útil
cur = col.find(
    {'size_x': {'$ne': '1'}}
)
for doc in cur:
    pprint(doc)

{'_id': ObjectId('5f090ccb7fdd86351b5840aa'),
 'common_action': 'Change the size of the letters or use the top character',
 'name': 'shift',
 'size_x': '3',
 'size_y': '1',
 'type': 'command'}
{'_id': ObjectId('5f090ccb7fdd86351b5840ab'),
 'common_action': 'ask for help',
 'description': 'no-sound icon with F1 writen on down-right, top-right has a '
                'led light that turns red when wound is off',
 'function': 'turns on/off the sound',
 'name': 'f1',
 'size_x': '0.8',
 'size_y': '0.5',
 'type': 'command'}


### Update
Eu reparei que tinha medido o shift errado, na verdade (pelas proporções de uma tecla média + duas pequenas) a medida é de x é 2.6

In [115]:
col.update(
    {'name': 'shift'}, 
    {'$set':
     {'size_x': '2.6'}
    }
)

<ipython-input-115-440a69f6813a>:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  col.update(


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [117]:
cur = col.find({'name': 'shift'})
pprint(cur.next())

{'_id': ObjectId('5f090ccb7fdd86351b5840aa'),
 'common_action': 'Change the size of the letters or use the top character',
 'name': 'shift',
 'size_x': '2.6',
 'size_y': '1',
 'type': 'command'}


### Delete

In [124]:
#apesar de haver dois com type command apenas o primeiro é excluído

col.delete_one({'character':'z'})
col.delete_one({'character':'x'})
col.delete_one({'type':'command'})
cur = col.find({})
for doc in cur:
    pprint(doc)

{'_id': ObjectId('5f090ccb7fdd86351b5840a8'),
 'character': 'c',
 'common_action': 'writes letter c',
 'name': 'ce',
 'shift': 'C',
 'size_x': '1',
 'size_y': '1',
 'type': 'letter'}
{'_id': ObjectId('5f090ccb7fdd86351b5840a9'),
 'character': ',',
 'common_action': 'writes a comma',
 'description': 'both characters on the left, comma lower than symbol of less '
                'than ',
 'name': 'comma',
 'shift': '<',
 'size_x': '1',
 'size_y': '1',
 'type': 'punctuation mark'}
{'_id': ObjectId('5f090ccb7fdd86351b5840ab'),
 'common_action': 'ask for help',
 'description': 'no-sound icon with F1 writen on down-right, top-right has a '
                'led light that turns red when wound is off',
 'function': 'turns on/off the sound',
 'name': 'f1',
 'size_x': '0.8',
 'size_y': '0.5',
 'type': 'command'}


In [129]:
# no caso como não há nada nos brackets isso será praticamente um drop
col.delete_many({})
cur = col.find({})
for doc in cur:
    pprint(doc)